# Filtracja bilateralna

## Konwolucja obrazu z filtrem o zadanych współczynnikach

Splot (konwolucję) obrazu wejściowego $I$ z filtrem $\psi$ dla ustalonego punktu obrazu $\mathbf{x}$ można przedstawić następująco:

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||)I(\mathbf{p})
\end{equation}

gdzie:
- $\hat{I}$ jest obrazem wynikowym (przefiltrowanym),
- $W_N = \sum_y \psi(y)$ jest parametrem normalizującym współczynniki filtra $\psi$,
- $||\cdot||$ jest odległością między punktami obrazu $\mathbf{x}$ i $\mathbf{p}$ według ustalonej metryki (np. norma $L_2$). Uwaga, proszę pamiętać, że zarówno $\mathbf{x}$, jak i $\mathbf{p}$ to współrzędne przestrzenne,
- $\eta(\mathbf{x})$ jest otoczeniem punktu $\mathbf{x}$.

Funkcja $\psi$ decyduje o charakterze filtracji. Dla filtru Gaussowskiego:

\begin{equation}
\psi(y) = G_{\delta_s}(y)
\end{equation}

gdzie: $G_{\delta_s}(y)$ jest funkcją Gaussa z parametrem skali $\delta_s$.

Opisaną powyżej filtrację realizowaliśmy w ramach ćwiczenia "Przetwarzanie wstępne. Filtracja kontekstowa."

## Filtracja bilateralna

Wadą klasycznego splotu jest brak adaptacji współczynników filtra do lokalnego otoczenia $\eta(\mathbf{x})$ filtrowanego punktu $\mathbf{x}$.
Oznacza to wykorzystanie tych samych współczynników filtra $\psi$ niezależnie od tego czy otoczenie jest względnie jednorodne lub zawiera krawędzie obiektów (w tym przypadku dochodzi do "rozmywania" krawędzi).
Filtracja bilateralna uwzględnia lokalne otoczenie filtrowanego punktu, w ten sposób, że parametry filtra zmieniają się w zależności od "wyglądu" otocznia.


Współczynniki filtra obliczane są na podstawie odległości filtrowanego punktu $\mathbf{x}$ od każdego punktu otoczenia $\mathbf{p}$ w dziedzinie przestrzennej obrazu (tak jak przy typowym filtrze np. Gaussa) oraz odległości punktów w przeciwdziedzinie obrazu (np. na podstawie różnicy w jasności pikseli dla obrazu w odcieniach szarości):

\begin{equation}
\hat{I}(\mathbf{x}) = \frac{1}{W_N}\sum_{\mathbf{p} \in \eta(\mathbf{x})} \psi(||\mathbf{p}-\mathbf{x}||) \gamma(|I(\mathbf{p}) - I(\mathbf{x})|) I(\mathbf{p})
\end{equation}
gdzie:
- $W_N$ jest współczynnikiem normalizującym filtr,
- $\gamma$ jest funkcją odległości w przeciwdziedzinie obrazu, np. $\gamma(y)=\exp(-\frac{y^2}{2\delta_r^2})$
- parametr $\delta_r$ jest utożsamiany z poziomem szumu w obrazie i należy go dobrać w sposób empiryczny.

Proszę chwilę zastanowić się nad powyższym równaniem, w szczególności nad funkcją $\gamma$. Proszę wyznaczyć, jaka będzie wartość funkcji dla pikseli podobnych (różnica 0, 1, 2), a skrajnie różnych (255, 200).

##  Realizacja ćwiczenia

### Wczytanie danych

1. Wczytaj dane z pliku *MR_data.mat*. W tym celu wykorzystaj funkcję `loadmat` z pakietu scipy:
        from scipy.io import loadmat
        mat = loadmat('MR_data.mat')

2. Wczytany plik zawiera 5 obrazów: *I_noisefree*, *I_noisy1*, *I_noisy2*, *I_noisy3* oraz *I_noisy4*. Odczytać je można w następujący sposób:
        Input = mat['I_noisy1']

3.Wyświetl wybrany obraz z pliku *MR_data.mat*. Zagadka - co to za obrazowanie medyczne?

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from scipy.io import loadmat
import math
import os

if not os.path.exists("MR_data.mat") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/07_Bilateral/MR_data.mat --no-check-certificate

#TODO Samodzielna
mat = loadmat('MR_data.mat')
Input = mat['I_noisefree']

plt.imshow(Input, cmap="gray")
plt.xticks([]), plt.yticks([])
plt.show()

### "Klasyczna" konwolucja

1. Zdefiniuj parametry filtra Gaussowskiego: rozmiar okna i wariancję $\delta_S$.
2. Oblicz współczynniki filtra na podstawie zdefiniowanych parametrów (najprościej w ramach podwójnej pętli for).
2. Sprawdź ich poprawność i zwizualizuj filtr (tak jak w ćwiczeniu pt. "Przetwarzanie wstępne. Filtracja kontekstowa.").
3. Wykonaj kopię obrazu wejściowego: `IConv = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentem tej funkcji są aktualnie przetwarzane okno i współczynniki filtra.
7. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
8. Wyświetl wynik filtracji.
9. Porównaj wynik z obrazem oryginalnym.

In [ ]:
#TODO Samodzielna
def fgaussian(size, sigma):
     m = n = size
     h, k = m//2, n//2
     x, y = np.mgrid[-h:h+1, -k:k+1]
     g = np.exp(-(x**2 + y**2)/(2*sigma**2))
     return g /g.sum()
    
def fgaussian2(size, sigma):
    kernel = np.zeros([size,size])
    x = size//2
    for i in range(size):
        for j in range(size):
            kernel[i, j] =  np.exp(-(pow(x-i,2) + pow(x-j,2))/(2*pow(sigma,2)))
    kernel = kernel / kernel.sum()
    return kernel
            

def mesh(fun, size):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    X = np.arange(-size//2, size//2, 1)
    Y = np.arange(-size//2, size//2, 1)
    X, Y = np.meshgrid(X, Y)
    Z = fun
    ax.plot_surface(X, Y, Z)
    plt.show()
    

#Porównanie fgaussian
kernel1 = fgaussian(5,0.9)
kernel2 = fgaussian2(5,0.9)
mesh(kernel1, 5)
mesh(kernel2, 5)

def process(context, kernel):
    size = kernel.shape[0]
    counter = 0
    weights = 0
    for i in range(size):
        for j in range(size):
            counter += context[i,j]*kernel[i,j]
            weights += kernel[i,j]
    return counter / weights

# Liczy się dłuższą chwilę 
def convolution(Img, kernel):
    k_size = kernel.shape[0]
    margin = k_size//2
    IConv = Img.copy()
    for i in range(margin,Img.shape[0]-margin):
        for j in range(margin,Img.shape[1]-margin):
            context = np.zeros([k_size,k_size])
            for c_i in range(-margin,margin+1):
                for c_j in range(-margin+1,margin+1):
                    context[c_i+margin,c_j+margin] = Img[i+c_i,j+c_j]
                    IConv[i,j] = process(context,kernel)
    return IConv

def convolution_filter(Img, kernel_size, sigma):
    return convolution(Img, fgaussian2(kernel_size, sigma))
            
IConv = convolution_filter(Input, 5, 0.9)

fig, axs = plt.subplots(1,2, figsize=(15,15))

axs[0].set_title("Oryginał")
axs[0].imshow(Input, 'gray')
axs[0].axis('off')

axs[1].set_title("Filtrowany")
axs[1].imshow(IConv, 'gray')
axs[1].axis('off')

plt.show()

### Filtracja bilateralna

1. Zdefiniuj dodatkowy parametr: wariancję $\delta_R$.
3. Wykonaj kopię obrazu wejściowego: `IBilateral = Input.copy()`
4. Wykonaj podwójną pętlę po obrazie. Pomiń ramkę, dla której nie jest zdefiniowany kontekst o wybranej wielkości.
5. W każdej iteracji stwórz dwuwymiarową tablicę zawierającą aktualny kontekst.
6. Napisz funkcję, która będzie obliczała nową wartość piksela.
Argumentami funkcji są aktualnie przetwarzane okno, współczynniki filtra gausowskiego (takie same jak wcześniej) i wariancja $\delta_R$.
7. Oblicz odległość w przeciwdziedzinie (dla wartości pikseli).
8. Oblicz funkcję Gaussa dla obliczonych odległości z zadanym parametrem.
9. Wykonaj normalizację obliczonych współczynników.
10. Obliczoną wartość przypisz do odpowiedniego piksela kopii obrazu wejściowego.
11. Wyświetl wynik filtracji.
12. Porównaj wynik z obrazem oryginalnym.

In [ ]:
def process_bi(context, kernel, sigma):
    size = kernel.shape[0]
    mid = size//2
    c_mid = context[mid,mid]
    counter = 0
    weights = 0
    new_cont = np.zeros(context.shape)
    
    for i in range(size):
        for j in range(size):
            new_cont[i, j] =  np.exp(-(pow(c_mid-context[i,j],2))/(2*pow(sigma,2)))
    
    for i in range(size):
        for j in range(size):
            counter += context[i,j]*kernel[i,j]*new_cont[i,j]
            weights += kernel[i,j]*new_cont[i,j]
    return counter / weights

# Liczy się dłuższą chwilę 
def bilateral(Img, kernel,sigma):
    k_size = kernel.shape[0]
    margin = k_size//2
    IBilateral = Img.copy()
    for i in range(margin,Img.shape[0]-margin):
        for j in range(margin,Img.shape[1]-margin):
            context = np.zeros([k_size,k_size])
            for c_i in range(-margin,margin+1):
                for c_j in range(-margin+1,margin+1):
                    context[c_i+margin,c_j+margin] = Img[i+c_i,j+c_j]
                    IBilateral[i,j] = process_bi(context,kernel,sigma)
    return IBilateral

def bilateral_filter(Img, kernel_size, sigma, sigma_r):
    return bilateral(Img, fgaussian2(kernel_size, sigma), sigma_r)

IBilateral = bilateral_filter(Input, 5, 0.7, 0.5)

fig, axs = plt.subplots(1,2, figsize=(15,15))

axs[0].set_title("Oryginał")
axs[0].imshow(Input, 'gray')    
axs[0].axis('off')

axs[1].set_title("Bi Filtrowany")
axs[1].imshow(IBilateral, 'gray')
axs[1].axis('off')

plt.show()